In [12]:
import pandas as pd
import numpy as np 
import random

In [6]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np

df = pd.read_csv('data/compas.data', 
                 parse_dates = ['DateOfBirth'])

removed_columns = [
    'Person_ID',
    'AssessmentID',
    'Case_ID',
    'LastName',
    'FirstName',
    'MiddleName',
    'Screening_Date',
    'RecSupervisionLevelText',
    'RawScore',
    'DecileScore',
    'IsCompleted',
    'IsDeleted'
]
df.drop(removed_columns, axis=1, inplace=True)


import datetime
import numpy as np
age = ((pd.Timestamp.now() - df['DateOfBirth']).dt.days / 365.25).astype(int)
age[age<0] = np.nan
df['age_'] = age


df.drop(df[df['ScoreText'].isnull()].index, inplace=True)
df.drop(df[df['age_'].isnull()].index, inplace=True)
df.drop(df[df['MaritalStatus']=='Unknown'].index, inplace=True)


age_bins = [0, 30, 100]
age_groups = pd.cut(df['age_'], bins=age_bins)
df['Age'] = age_groups
num_groups = len(df['Age'].cat.categories)
df['Age'] = df['Age'].cat.rename_categories(range(num_groups))


df.Sex_Code_Text = pd.Categorical(df.Sex_Code_Text)
df['Sex_Code_Text'] = df.Sex_Code_Text.cat.codes

df["Ethnic_Code_Raw"] = df["Ethnic_Code_Text"]
df.Ethnic_Code_Text = pd.Categorical(df.Ethnic_Code_Text)
df['Ethnic_Code_Text'] = df.Ethnic_Code_Text.cat.codes


df.MaritalStatus = pd.Categorical(df.MaritalStatus)
df['MaritalStatus'] = df.MaritalStatus.cat.codes

df.CustodyStatus = pd.Categorical(df.CustodyStatus)
df['CustodyStatus'] = df.CustodyStatus.cat.codes

df.LegalStatus = pd.Categorical(df.LegalStatus)
df['LegalStatus'] = df.LegalStatus.cat.codes

df['ScoreText_'] = -1
mask = df['ScoreText']=='High'
df.loc[mask, 'ScoreText_'] = 0
df.loc[~mask, 'ScoreText_'] = 1


df.drop(['DisplayText','ScoreText','Agency_Text', 'AssessmentType', 'ScaleSet_ID', 'ScaleSet', 'AssessmentReason', 'Language'], axis=1, inplace=True)
df.drop(['DateOfBirth', 'age_', 'Scale_ID'], axis=1, inplace=True)
df = df[df['Age'].notna()]


df.to_csv('data/compass_categorized.csv', index = False)


/tmp/ipykernel_357/2355448443.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('data/compas.data',


In [7]:
raw = pd.read_csv('data/compass_categorized.csv')

In [8]:
raw.head()

,Sex_Code_Text,Ethnic_Code_Text,LegalStatus,CustodyStatus,MaritalStatus,RecSupervisionLevel,Age,Ethnic_Code_Raw,ScoreText_
0,1,4,5,0,4,1,1,Caucasian,1
1,1,4,5,0,4,1,1,Caucasian,1
2,1,4,5,0,4,1,1,Caucasian,1
3,1,4,5,0,1,1,1,Caucasian,1
4,1,4,5,0,1,1,1,Caucasian,1


In [9]:
raw.shape

(42005, 9)

To create Unit test data 

In [10]:
for col in raw: 
    print(f"{col} , {raw[col].unique()}")

Sex_Code_Text , [1 0]
Ethnic_Code_Text , [4 1 5 8 3 0 6 7 2]
LegalStatus , [5 4 0 2 6 3 1]
CustodyStatus , [0 4 2 5 3 1]
MaritalStatus , [4 1 3 0 2 5]
RecSupervisionLevel , [1 4 2 3]
Age , [1 0]
Ethnic_Code_Raw , ['Caucasian' 'African-American' 'Hispanic' 'Other' 'Asian' 'African-Am'
 'Native American' 'Oriental' 'Arabic']
ScoreText_ , [1 0]


In [14]:
data = {
    "Sex_Code_Text": [1, 0],
    "Ethnic_Code_Text": [4, 1, 5, 8, 3, 0, 6, 7, 2],
    "LegalStatus": [5, 4, 0, 2, 6, 3, 1],
    "CustodyStatus": [0, 4, 2, 5, 3, 1],
    "MaritalStatus": [4, 1, 3, 0, 2, 5],
    "RecSupervisionLevel": [1, 4, 2, 3],
    "Age": [1, 0],
    "Ethnic_Code_Raw": [
        'Caucasian', 'African-American', 'Hispanic', 'Other', 'Asian',
        'African-Am', 'Native American', 'Oriental', 'Arabic'
    ],
    "ScoreText_": [1, 0]
}

# Generate 1000 rows of random data
rows = 1000
random_data = {
    column: [random.choice(values) for _ in range(rows)] for column, values in data.items()
}

# Create a DataFrame
df = pd.DataFrame(random_data)

# Save to a CSV file
file_path = "unit_test_data/mock_compass_1000.csv"
df.to_csv(file_path, index=False)